In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [3]:
# Train data 
train = torchvision.datasets.CIFAR10('./CIFAR10',
                                     train=True,
                                     download=False,
                                     transform=transform)
# Test data 

test = torchvision.datasets.CIFAR10('./CIFAR10',
                                     train=False,
                                     download=False,
                                     transform=transform)

In [4]:
from torch.utils.data import DataLoader

# Load data into DataLoader
trainloader = DataLoader(train,64,shuffle=True)
testloader = DataLoader(test,64,shuffle=True)

In [5]:
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [6]:
# Make a Convolutional neural network
class ConvNet(nn.Module):
    def __init__(self) -> None:
        # Don't forget 
        super().__init__() 
        # Conv layers
        self.pool = nn.MaxPool2d(2, 2)
        self.cl1 = nn.Conv2d(3,32,3) # 3 inputs for rgb
        self.cl2 = nn.Conv2d(32,64,3)
        self.cl3 = nn.Conv2d(64,64,3)
        #Fully connected layers
        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        # After convolutions & pooling, size calculation:
        # Input: 32x32 -> conv1 -> 30x30 -> pool -> 15x15
        # -> conv2 -> 13x13 -> pool -> 6x6
        # -> conv3 -> 4x4 with 64 channels
        self.fc2 = nn.Linear(64, 10)
        self.relu = nn.ReLU()
    def forward(self,x):
        # P for poling
        x = self.pool(self.relu(self.cl1(x)))
        x = self.pool(self.relu(self.cl2(x)))
        x = self.relu(self.cl3(x))
        x = x.view(-1, 64 * 4 * 4)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


    

In [ ]:
# At the start of your script, add this device setup and dont be stupid use gpu:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model,loss_function,optimaizer,EPOCHS=10):
    model.train()
    for epoch in tqdm(range(EPOCHS)):
        running_loss = 0.0
        for i,(images,labels) in enumerate(trainloader):
            images = images.to(device)
            labels = labels.to(device)
            

            optimaizer.zero_grad()

            outputs = model(images)
            loss = loss_function(outputs,labels)
            loss.backward()
            optimaizer.step()

            running_loss += loss.item()
            
            if i % 100 == 99:
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0

In [ ]:
def test_model(model):
    model.eval()
    correct = 0
    total =0 
    class_correct = [0] * 10
    class_total = [0] * 10
    with torch.no_grad():  # Disable gradient computation
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Per-class accuracy
            c = (predicted == labels)
            for i in tqdm(range(len(labels))):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

In [13]:
model = ConvNet().to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_model(model,loss_function=loss,optimaizer=optimizer,)
test_model(model)

[1,   100] loss: 2.007
[1,   200] loss: 1.715
[1,   300] loss: 1.558
[1,   400] loss: 1.481
[1,   500] loss: 1.425
[1,   600] loss: 1.372
[1,   700] loss: 1.317
[2,   100] loss: 1.261
[2,   200] loss: 1.244
[2,   300] loss: 1.192
[2,   400] loss: 1.169
[2,   500] loss: 1.126
[2,   600] loss: 1.143
[2,   700] loss: 1.106
[3,   100] loss: 1.033
[3,   200] loss: 1.039
[3,   300] loss: 1.002
[3,   400] loss: 0.994
[3,   500] loss: 0.983
[3,   600] loss: 0.952
[3,   700] loss: 0.971
[4,   100] loss: 0.900
[4,   200] loss: 0.879
[4,   300] loss: 0.885
[4,   400] loss: 0.876
[4,   500] loss: 0.865
[4,   600] loss: 0.883
[4,   700] loss: 0.870
[5,   100] loss: 0.794
[5,   200] loss: 0.796
[5,   300] loss: 0.788
[5,   400] loss: 0.818
[5,   500] loss: 0.798
[5,   600] loss: 0.804
[5,   700] loss: 0.810
[6,   100] loss: 0.712
[6,   200] loss: 0.722
[6,   300] loss: 0.722
[6,   400] loss: 0.748
[6,   500] loss: 0.715
[6,   600] loss: 0.746
[6,   700] loss: 0.759
[7,   100] loss: 0.670
[7,   200] 

UnboundLocalError: cannot access local variable 'correct' where it is not associated with a value

In [14]:
net = ConvNet()

In [15]:
torch.save(net, 'model.pth')

torch.save(net.state_dict(), 'model_weights.pth')
